#### Packages

In [1]:
from gurobipy import *
import pandas as pd
import numpy as np
import grblogtools as glt

#### Read Data

In [2]:
store_info = pd.read_excel('OR_hw01_data.xlsx', 'Store')
stores = range(len(store_info['Store ID']))
expected_daily_demand = store_info['Expected Daily Demand (unit)']
store_xcoord = store_info['x-coordinate (km)']
store_ycoord = store_info['y-coordinate (km)']

dc_info = pd.read_excel('OR_hw01_data.xlsx', 'DC')
dcs = range(len(dc_info['Location ID']))
maintenance_costs = dc_info['Maintenance Cost ($/unit)']
construction_costs = dc_info['Construction Cost ($)']
dc_xcoord = dc_info['x-coordinate (km)']
dc_ycoord = dc_info['y-coordinate (km)']
dc_maxscale = dc_info['Maximum Scale (unit)']

S = 1 # replenishment cost

#### Model 1(a): min TC (Multiple sourcing)

In [3]:
eg1a = Model("eg1a")

#-------- Add variables as a list ---------#
# vj = 1 if a DC is built at loc j
v = []
for j in dcs:
    v.append(eg1a.addVar(lb=0, vtype = GRB.BINARY, name = "v" + str(j+1)))
    
# sj = the scale level of a DC built at loc j
s = []
for j in dcs:
    s.append(eg1a.addVar(lb=0, vtype = GRB.INTEGER, name = "s" + str(j+1)))

# rij = the amount of products replenished by DCj to store i
r = []
for i in stores:
    r.append([])
    for j in dcs:
        r[i].append(eg1a.addVar(lb = 0, vtype = GRB.INTEGER, name = "r" + str(i+1) + str(j+1)))
        

# Manhattan distancebetween store i and DCj on x-axis
wx = []
for i in stores:
    wx.append([])
    for j in dcs:
        wx[i].append(eg1a.addVar(lb = 0, vtype = GRB.INTEGER, name = "wx" + str(i+1) + str(j+1)))
# Manhattan distancebetween store i and DCj on y-axis
wy = []
for i in stores:
    wy.append([])
    for j in dcs:
        wy[i].append(eg1a.addVar(lb = 0, vtype = GRB.INTEGER, name = "wy" + str(i+1) + str(j+1)))

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-29


##### Objective Fucntion

In [4]:
eg1a.setObjective(
    quicksum(v[j] * construction_costs[j] for j in dcs) \
    + quicksum(v[j] * s[j] * maintenance_costs[j] for j in dcs) \
    + quicksum(quicksum(S * r[i][j] * (wx[i][j] + wy[i][j]) for j in dcs) for i in stores)\
    ,GRB.MINIMIZE
)

##### Constraints

In [ ]:
eg1a.addConstrs((quicksum(v[j] * r[i][j] for j in dcs) == expected_daily_demand[i] for i in stores), "demand_fulfillment")
eg1a.addConstrs((quicksum(r[i][j] for i in stores) == s[j] for j in dcs), "DC scale lv == its total replenishment q")
eg1a.addConstrs((s[j] <= v[j] * dc_maxscale[j] for j in dcs), "DC scale lv <= its max scale lv")
eg1a.addConstrs((quicksum(r[i][j] for i in stores) >= 0 for j in dcs), "replenishment >= 0")

#eg1a.addConstrs((quicksum(r[i][j] for i in stores) <= v[j] * 20000 for j in dcs), "replenishment amount exists when vj=1")
eg1a.addConstrs((wx[i][j] >= dc_xcoord[j] - store_xcoord[i] for i in stores for j in dcs), "dis(dc-store), x-axis")
eg1a.addConstrs((wx[i][j] >= store_xcoord[i] - dc_xcoord[j] for j in dcs for i in stores), "dis(store-dc), x-axis")
eg1a.addConstrs((wy[i][j] >= dc_ycoord[j] - store_ycoord[i] for i in stores for j in dcs), "dis(dc-store), y-axis")
eg1a.addConstrs((wy[i][j] >= store_ycoord[i] - dc_ycoord[j] for j in dcs for i in stores), "dis(store-dc), y-axis")

##### Optimize

In [ ]:
eg1a.optimize()

##### 1(a) Results

In [7]:
print("1(a) Results | Multiple sourcing: ")

for j in dcs:
    print(v[j].varName, '=', int(v[j].x))

"""
for j in dcs:
    print("DC" + str(j+1), "\t", end="")
    for i in stores:
        if len(str(r[i][j].x)) < 11:
            print(int(r[i][j].x), "\t", end="")
        else:
            print(int(r[i][j].x), "", end="")
    print("")
"""

print("DC's scale: ")
for j in dcs:
    count = 0
    print("DC" + str(j+1), "\t", end="")
    for i in stores:
        count += int(r[i][j].x)
    print(count)

print("z* = ", eg1a.ObjVal)

1(a) Results | Multiple sourcing: 
v1 = 1
v2 = 1
v3 = 1
v4 = 1
v5 = 1
v6 = 1
v7 = 1
v8 = 0
v9 = 1
v10 = 1
DC's scale: 
DC1 	3124
DC2 	1240
DC3 	910
DC4 	1498
DC5 	1389
DC6 	2592
DC7 	1218
DC8 	0
DC9 	1606
DC10 	1584
z* =  486706.0


#### Model 1(b): min TC (Single sourcing)

In [8]:
eg1b = Model("eg1b")
eg1b.Params.LogFile = "eg1b.log" 
results = glt.parse("eg1b*.log")
summary = results.summary()
nodelog_progress = results.progress("nodelog")


#-------- Add variables as a list ---------#
# vj = 1 if a DC is built at loc j
v = []
for j in dcs:
    v.append(eg1b.addVar(lb=0, vtype = GRB.BINARY, name = "v" + str(j+1)))

# sj = the scale level of a DC built at loc j
s = []
for j in dcs:
    s.append(eg1b.addVar(lb=0, vtype = GRB.INTEGER, name = "s" + str(j+1)))

# rij = the amount of products replenished by DCj to store i
r = []
for i in stores:
    r.append([])
    for j in dcs:
        r[i].append(eg1b.addVar(lb = 0, vtype = GRB.INTEGER, name = "r" + str(i+1) + str(j+1)))
        
# zij = 1 if products are replenished by DCj to store i (rij>0)
z = []
for i in stores:
    z.append([])
    for j in dcs:
        z[i].append(eg1b.addVar(lb = 0, vtype = GRB.BINARY, name = "z" + str(i+1) + str(j+1)))

# Manhattan distancebetween store i and DCj on x-axis
wx = []
for i in stores:
    wx.append([])
    for j in dcs:
        wx[i].append(eg1b.addVar(lb = 0, vtype = GRB.INTEGER, name = "wx" + str(i+1) + str(j+1)))
# Manhattan distancebetween store i and DCj on y-axis
wy = []
for i in stores:
    wy.append([])
    for j in dcs:
        wy[i].append(eg1b.addVar(lb = 0, vtype = GRB.INTEGER, name = "wy" + str(i+1) + str(j+1)))

Set parameter LogFile to value "eg1b.log"


##### Objective Function

In [9]:
eg1b.setObjective(
    quicksum(v[j] * construction_costs[j] for j in dcs) \
    + quicksum(v[j] * s[j] * maintenance_costs[j] for j in dcs) \
    + quicksum(quicksum(r[i][j] * (wx[i][j] + wy[i][j]) for i in stores) for j in dcs)\
    ,GRB.MINIMIZE
)

##### Constraints

In [ ]:
eg1b.addConstrs((quicksum(z[i][j] * r[i][j] for j in dcs) == expected_daily_demand[i] for i in stores), "demand_fulfillment")
eg1b.addConstrs((quicksum(z[i][j] for j in dcs) == 1 for i in stores), "每間商店都只能被一個DC去補")

eg1b.addConstrs((quicksum(r[i][j] for i in stores) == s[j] for j in dcs), "DC scale lv == its total replenishment q")
eg1b.addConstrs((s[j] <= v[j] * dc_maxscale[j] for j in dcs), "DC scale lv <= its max scale lv")
eg1b.addConstrs((quicksum(r[i][j] for i in stores) >= 0 for j in dcs), "replenishment amount >= 0")

eg1b.addConstrs((wx[i][j] >= dc_xcoord[j] - store_xcoord[i] for i in stores for j in dcs), "dis(dc-store), x-axis")
eg1b.addConstrs((wx[i][j] >= store_xcoord[i] - dc_xcoord[j] for j in dcs for i in stores), "dis(store-dc), x-axis")
eg1b.addConstrs((wy[i][j] >= dc_ycoord[j] - store_ycoord[i] for i in stores for j in dcs), "dis(dc-store), y-axis")
eg1b.addConstrs((wy[i][j] >= store_ycoord[i] - dc_ycoord[j] for j in dcs for i in stores), "dis(store-dc), y-axis")

##### Optimize

In [ ]:
eg1b.optimize()

##### Results

In [12]:
print("1(b) Results | Single sourcing: ")

for j in dcs:
    print(v[j].varName, '=', int(v[j].x))

"""
replenish_plan = []   
for j in dcs:
    print("DC" + str(j+1), "\t", end="")
    for i in stores:
        if len(str(r[i][j].x)) < 11:
            print(int(r[i][j].x), "\t", end="")
        else:
            print(int(r[i][j].x), "", end="")
    print("")
"""

print("DC's scale: ")
for j in dcs:
    count = 0
    print("DC" + str(j+1), "\t", end="")
    for i in stores:
        count += int(r[i][j].x)
    print(count)

print("z* = ", eg1b.ObjVal)

1(b) Results | Single sourcing: 
v1 = 1
v2 = 1
v3 = 1
v4 = 1
v5 = 1
v6 = 1
v7 = 1
v8 = 0
v9 = 1
v10 = 1
DC's scale: 
DC1 	3119
DC2 	1234
DC3 	910
DC4 	1493
DC5 	1437
DC6 	2580
DC7 	1215
DC8 	0
DC9 	1598
DC10 	1575
z* =  493168.0


#### Model 2: min the maximum distance

In [13]:
eg2 = Model("eg2")
B = int #cost_constraint

#-------- Add variables as a list ---------#

# vj = 1 if a DC is built at loc j
v = []
for j in dcs:
    v.append(eg2.addVar(lb=0, vtype = GRB.BINARY, name = "v" + str(j+1)))

# sj = the scale level of a DC built at loc j
s = []
for j in dcs:
    s.append(eg2.addVar(lb=0, vtype = GRB.INTEGER, name = "s" + str(j+1)))

# rij = the amount of products replenished by DCj to store i
r = []
for i in stores:
    r.append([])
    for j in dcs:
        r[i].append(eg2.addVar(lb = 0, vtype = GRB.INTEGER, name = "r" + str(i+1) + str(j+1)))

# Manhattan distancebetween store i and DCj on x-axis
wx = []
for i in stores:
    wx.append([])
    for j in dcs:
        wx[i].append(eg2.addVar(lb = 0, vtype = GRB.INTEGER, name = "wx" + str(i+1) + str(j+1)))
# Manhattan distancebetween store i and DCj on y-axis
wy = []
for i in stores:
    wy.append([])
    for j in dcs:
        wy[i].append(eg2.addVar(lb = 0, vtype = GRB.INTEGER, name = "wy" + str(i+1) + str(j+1)))


# wij=1 if the DCj is the closest to store i
w = []
for i in stores:
    w.append([])
    for j in dcs:
        w[i].append(eg2.addVar(lb = 0, vtype = GRB.BINARY, name = "w" + str(i+1) + str(j+1)))

d = []
for i in stores:
    d.append(eg2.addVar(lb = 0, vtype = GRB.INTEGER, name = "d"))
B = eg2.addVar(lb = 0, vtype = GRB.INTEGER, name = "B")
max_distance = eg2.addVar(lb = 0, vtype = GRB.INTEGER, name = "max_distance")

##### Objective Function

In [14]:
eg2.setObjective(max_distance,GRB.MINIMIZE)

##### Constraints

In [ ]:
eg2.addConstrs((quicksum(v[j] * r[i][j] for j in dcs) == expected_daily_demand[i] for i in stores), "demand_fulfillment")
eg2.addConstrs((quicksum(r[i][j] for i in stores) == s[j] for j in dcs), "DC scale lv == its total replenishment q")
eg2.addConstrs((s[j] <= v[j] * dc_maxscale[j] for j in dcs), "DC scale lv <= its max scale lv")
eg2.addConstrs((quicksum(r[i][j] for i in stores) >= 0 for j in dcs), "replenishment >= 0")

eg2.addConstr(quicksum(construction_costs[j] + maintenance_costs[j] * s[j] for j in dcs) <= B, "TC cannot > B")
eg2.addConstrs((quicksum(w[i][j] for j in dcs) == 1 for i in stores), "每間store都有一個最近的DC")
eg2.addConstrs((w[i][j] <= v[j] for i in stores for j in dcs), "要確定那間DC有蓋")
eg2.addConstrs((max_distance >= quicksum(w[i][j] * (wx[i][j] + wy[i][j]) for j in dcs) for i in stores), "每間store到最近的DC的最遠距離")
eg2.addConstrs((wx[i][j] >= dc_xcoord[j] - store_xcoord[i] for i in stores for j in dcs), "dis(dc-store), x-axis")
eg2.addConstrs((wx[i][j] >= store_xcoord[i] - dc_xcoord[j] for j in dcs for i in stores), "dis(store-dc), x-axis")
eg2.addConstrs((wy[i][j] >= dc_ycoord[j] - store_ycoord[i] for i in stores for j in dcs), "dis(dc-store), y-axis")
eg2.addConstrs((wy[i][j] >= store_ycoord[i] - dc_ycoord[j] for j in dcs for i in stores), "dis(store-dc), y-axis")

##### Optimize

In [ ]:
eg2.optimize()

##### Results

In [17]:
print("3 Results | Multiple sourcing: ")

for j in dcs:
    print(v[j].varName, '=', int(v[j].x))

"""
replenish_plan = []   
for j in dcs:
    print("DC" + str(j+1), "\t", end="")
    for i in stores:
        if len(str(r[i][j].x)) < 11:
            print(int(r[i][j].x), "\t", end="")
        else:
            print(int(r[i][j].x), "", end="")
    print("")
"""

print("DC's scale: ")
for j in dcs:
    count = 0
    print("DC" + str(j+1), "\t", end="")
    for i in stores:
        count += int(r[i][j].x)
    print(count)

print("z* = ", eg2.ObjVal)

3 Results | Multiple sourcing: 
v1 = 1
v2 = 1
v3 = 1
v4 = 1
v5 = 1
v6 = 1
v7 = 1
v8 = 1
v9 = 1
v10 = 1
DC's scale: 
DC1 	3124
DC2 	1240
DC3 	910
DC4 	1498
DC5 	1027
DC6 	2304
DC7 	1218
DC8 	650
DC9 	1606
DC10 	1584
z* =  51.0


#### Model 3: Dual Sourcing

In [18]:
eg3 = Model("eg3")

#-------- Add variables as a list ---------#
# vj = 1 if a DC is built at loc j
v = []
for j in dcs:
    v.append(eg3.addVar(lb=0, vtype = GRB.BINARY, name = "v" + str(j+1)))
    
# sj = the scale level of a DC built at loc j
s = []
for j in dcs:
    s.append(eg3.addVar(lb=0, vtype = GRB.INTEGER, name = "s" + str(j+1)))

# rij = the amount of products replenished by DCj to store i
r = []
for i in stores:
    r.append([])
    for j in dcs:
        r[i].append(eg3.addVar(lb = 0, vtype = GRB.INTEGER, name = "r" + str(i+1) + str(j+1)))
        
# zij = 1 if products are replenished by DCj to store i (rij>0)
z = []
for i in stores:
    z.append([])
    for j in dcs:
        z[i].append(eg3.addVar(lb = 0, vtype = GRB.BINARY, name = "z" + str(i+1) + str(j+1)))

# Manhattan distancebetween store i and DCj on x-axis
wx = []
for i in stores:
    wx.append([])
    for j in dcs:
        wx[i].append(eg3.addVar(lb = 0, vtype = GRB.INTEGER, name = "wx" + str(i+1) + str(j+1)))
# Manhattan distancebetween store i and DCj on y-axis
wy = []
for i in stores:
    wy.append([])
    for j in dcs:
        wy[i].append(eg3.addVar(lb = 0, vtype = GRB.INTEGER, name = "wy" + str(i+1) + str(j+1)))

##### Objective Function

In [19]:
eg3.setObjective(
    quicksum(v[j] * construction_costs[j] for j in dcs) \
    + quicksum(v[j] * s[j] * maintenance_costs[j] for j in dcs) \
    + quicksum(quicksum(S * r[i][j] * (wx[i][j] + wy[i][j]) for j in dcs) for i in stores)\
    ,GRB.MINIMIZE
)

##### Constraints

In [ ]:
eg3.addConstrs((quicksum(z[i][j] * r[i][j] for j in dcs) == expected_daily_demand[i] for i in stores), "demand_fulfillment")
eg3.addConstrs((quicksum(z[i][j] for j in dcs) <= 2 for i in stores), "每間商店都最多只能被2個DC去補")
eg3.addConstrs((quicksum(r[i][j] for i in stores) == s[j] for j in dcs), "DC scale lv == its total replenishment q")
eg3.addConstrs((s[j] <= v[j] * dc_maxscale[j] for j in dcs), "DC scale lv <= its max scale lv")
eg3.addConstrs((quicksum(r[i][j] for i in stores) >= 0 for j in dcs), "replenishment >= 0")

#eg1a.addConstrs((quicksum(r[i][j] for i in stores) <= v[j] * 20000 for j in dcs), "replenishment amount exists when vj=1")
eg3.addConstrs((wx[i][j] >= dc_xcoord[j] - store_xcoord[i] for i in stores for j in dcs), "dis(dc-store), x-axis")
eg3.addConstrs((wx[i][j] >= store_xcoord[i] - dc_xcoord[j] for j in dcs for i in stores), "dis(store-dc), x-axis")
eg3.addConstrs((wy[i][j] >= dc_ycoord[j] - store_ycoord[i] for i in stores for j in dcs), "dis(dc-store), y-axis")
eg3.addConstrs((wy[i][j] >= store_ycoord[i] - dc_ycoord[j] for j in dcs for i in stores), "dis(store-dc), y-axis")

##### Optimize

In [ ]:
eg3.optimize()

##### Results

In [22]:
print("4 Results | Dual sourcing: ")

for j in dcs:
    print(v[j].varName, '=', int(v[j].x))

"""
replenish_plan = []   
for j in dcs:
    print("DC" + str(j+1), "\t", end="")
    for i in stores:
        if len(str(r[i][j].x)) < 11:
            print(int(r[i][j].x), "\t", end="")
        else:
            print(int(r[i][j].x), "", end="")
    print("")
"""

print("DC's scale: ")
for j in dcs:
    count = 0
    print("DC" + str(j+1), "\t", end="")
    for i in stores:
        count += int(r[i][j].x)
    print(count)

print("z* = ", eg3.ObjVal)

4 Results | Dual sourcing: 
v1 = 1
v2 = 1
v3 = 1
v4 = 1
v5 = 1
v6 = 1
v7 = 1
v8 = 0
v9 = 1
v10 = 1
DC's scale: 
DC1 	3123
DC2 	1240
DC3 	910
DC4 	1498
DC5 	1389
DC6 	2592
DC7 	1218
DC8 	0
DC9 	1606
DC10 	1584
z* =  486706.0


#### Visualization

In [23]:
glt.plot(results.summary())

interactive(children=(Dropdown(description='x', options=('Platform', 'Time', 'PhysicalCores', 'LogicalProcesso…